In [1]:
import pandas as pd
import numpy as np
from datamodel import TradingState, Listing, OrderDepth

In [2]:
data = pd.read_csv('results/zero-trades.csv', delimiter=';')

In [3]:
data.head(10)

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-1,0,PEARLS,9998,1,9995.0,30.0,NaN,NaN,10005,30,NaN,NaN,NaN,NaN,10001.5,0.0
1,-1,0,BANANAS,4946,1,4945.0,30.0,NaN,NaN,4952,30,NaN,NaN,NaN,NaN,4949.0,0.0
2,-1,100,PEARLS,9996,1,9995.0,30.0,NaN,NaN,10002,6,10004.0,1.0,10005.0,30.0,9999.0,0.0
3,-1,100,BANANAS,4945,31,NaN,NaN,NaN,NaN,4950,7,4952.0,31.0,NaN,NaN,4947.5,0.0
4,-1,200,BANANAS,4945,22,NaN,NaN,NaN,NaN,4951,1,4952.0,21.0,NaN,NaN,4948.0,0.0
5,-1,200,PEARLS,9996,1,9995.0,21.0,NaN,NaN,10004,1,10005.0,21.0,NaN,NaN,10000.0,0.0
6,-1,300,PEARLS,9996,2,9995.0,23.0,NaN,NaN,9998,3,10004.0,2.0,10005.0,23.0,9997.0,0.0
7,-1,300,BANANAS,4945,25,NaN,NaN,NaN,NaN,4952,25,NaN,NaN,NaN,NaN,4948.5,0.0
8,-1,400,PEARLS,9998,5,9996.0,2.0,9995.0,23.0,10004,2,10005.0,23.0,NaN,NaN,10001.0,0.0
9,-1,400,BANANAS,4946,5,4945.0,25.0,NaN,NaN,4952,25,NaN,NaN,NaN,NaN,4949.0,0.0


In [4]:
datad = data.to_dict("records")

In [5]:
trading_states_d = {}
for d in datad:
    timestamp = d['timestamp']
    product = d['product']
    buy_orders = {}
    sell_orders = {}
    # bids
    for i in range(1,4):
        price = d['bid_price_'+str(i)]
        volume = d['bid_volume_'+str(i)]
        
        buy_orders[price] = volume
    # asks
    for i in range(1,4):
        price = d['ask_price_'+str(i)]
        volume = d['ask_volume_'+str(i)]
        sell_orders[price] = volume
    
    listing = Listing(
        product,
        product,
        product
    )

    order_depth = OrderDepth(
        buy_orders,
        sell_orders,
        d['mid_price']
    )


    if timestamp not in trading_states_d:
        trading_states_d[timestamp] = TradingState(
            timestamp,
            {},
            {},
            {},
            {},
            {},
            {}
        )

    trading_states_d[timestamp].listings[product] = listing
    trading_states_d[timestamp].order_depths[product] = order_depth


states = list(trading_states_d.items())
states[0][1].order_depths['PEARLS'].buy_orders

{9998: 1, 9995.0: 30.0, nan: nan}

In [13]:
def wavg_bid_price(states, windows, product):
    ret = []
    mw = max(windows)
    ds = states[-mw:]
    wsum = 0
    vsum = 0
    for wi, (_, d) in enumerate(ds[::-1]):
        for p, v in d.order_depths[product].buy_orders.items():
            if not np.isnan(p):
                wsum += p*v
                vsum += v
        if wi+1 in windows:
            ret.append(wsum/vsum)

    while len(ret) < len(windows):
        ret.append(np.nan)
    return ret

print(wavg_bid_price(states, [1, 5, 10, 15], "PEARLS"))

def wavg_ask_price(states, windows, product):
    ret = []
    mw = max(windows)
    ds = states[-mw:]
    wsum = 0
    vsum = 0
    for wi, (_, d) in enumerate(ds[::-1]):
        for p, v in d.order_depths[product].sell_orders.items():
            if not np.isnan(p):
                wsum += p*v
                vsum += v
        if wi+1 in windows:
            ret.append(wsum/vsum)
    while len(ret) < len(windows):
        ret.append(np.nan)
    return ret

print(wavg_ask_price(states, [1, 5, 10, 15], "PEARLS"))

def avg_mid_price(states, windows, product):
    ret = []
    mw = max(windows)
    ds = states[-mw:]
    wsum = 0
    vsum = 0
    for wi, (_, d) in enumerate(ds[::-1]):
        wsum += d.order_depths[product].mid_price
        vsum += 1
        if wi+1 in windows:
            ret.append(wsum/vsum)
    while len(ret) < len(windows):
        ret.append(np.nan)
    return ret

print(avg_mid_price(states, [1, 5, 10, 15], "PEARLS"))

def volume_diff(states, windows, product):
    ret = []
    mw = max(windows)
    ds = states[-mw:]
    vsum = 0
    for wi, (_, d) in enumerate(ds[::-1]):
        for p, v in d.order_depths[product].buy_orders.items():
            if not np.isnan(p):
                vsum -= v
        for p, v in d.order_depths[product].sell_orders.items():
            if not np.isnan(p):
                vsum += v
        if wi+1 in windows:
            ret.append(vsum)
    while len(ret) < len(windows):
        ret.append(np.nan)
    return ret

print(volume_diff(states, [1, 5, 10, 15], "PEARLS"))

def best_prices(states, windows, product):
    ret = []
    mw = max(windows)
    ds = states[-mw:]
    asks = []
    bids = []
    for wi, (_, d) in enumerate(ds[::-1]):
        for p, _ in d.order_depths[product].buy_orders.items():
            if not np.isnan(p):
                bids.append(p)
        for p, _ in d.order_depths[product].sell_orders.items():
            if not np.isnan(p):
                asks.append(p)
        if wi+1 in windows:
            ret.append(max(bids))
            ret.append(min(asks))
    while len(ret) < len(windows):
        ret.append(np.nan)
    return ret

print(best_prices(states, [1, 5], "PEARLS"))

[9995.79411764706, 9995.265734265735, 9995.253472222223, 9995.217494089835]
[10005.0, 10004.866666666667, 10004.909420289856, 10004.885922330097]
[10001.5, 10000.3, 10000.5, 10000.3]
[-9.0, -8.0, -12.0, -11.0]
[9998, 10005, 10002, 9998]


In [23]:
products = ['PEARLS', 'BANANAS']
inds = [wavg_bid_price, wavg_ask_price, avg_mid_price, volume_diff, best_prices]
xss = {}
windows = [1, 2, 4, 8, 10, 15]

for product in products:
    xs = []
    for i in range(len(states)):
        x = []
        ws = states[:i]
        for ind in inds:
            x.extend(ind(ws, windows, product))
        xs.append(np.array(x))
    xss[product] = np.array(xs)

/tmp/ipykernel_3268088/1356443796.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  xss[product] = np.array(xs)


In [24]:
for i, product_i in enumerate(products):
    for product_j in products[i:]:
        xss[product_i+product_j] = xss[product_i] - xss[product_j]

In [129]:
def compute_gt(states, product, margins, volumes, windows, shortsell=False):
    ret = {}
    mw = max(windows)
    ds = states[:mw]
    # Purchases
    infrom = ds[0][1].order_depths[product].sell_orders
    if shortsell:
        infrom = ds[0][1].order_depths[product].buy_orders
    purchases = {}
    psum = 0
    vsum = 0
    tbuy = 0
    for p, v in infrom.items():
        if tbuy > max(volumes):
            break
        if not np.isnan(p):
            for pv in range(1, int(v)+1):
                if tbuy > max(volumes):
                    break
                psum += p
                vsum += 1
                tbuy += 1
                if tbuy in volumes:
                    purchases[tbuy] = psum/vsum
                    psum = 0
                    vsum = 0  
    # Sells  
    for margin in margins:
        ret[margin] = {1:{}}
        currw = ret[margin][1]
        currp = purchases[1]   
        tsold = 0
        for wi, (_, d) in enumerate(ds):
            outto = d.order_depths[product].buy_orders
            if shortsell:
                outto = d.order_depths[product].sell_orders
            if wi+1 in windows:
                ret[margin][wi+1] = {}
                currw = ret[margin][wi+1] = {
                    volume: 1 if tsold >= volume else 0 for volume in volumes 
                }
            tsum = 0
            vsum = 0
            avg = 0
            for p, v in outto.items():
                if not np.isnan(p):
                    for _ in range(int(v)):
                        tsum += p
                        vsum += 1
                        avg = tsum/vsum
                        if (
                            ((avg + margin) <= currp   and not shortsell) or
                            ((currp - margin) <= avg and shortsell)
                        ):
                            break
                        tsold += 1

                        if tsold in volumes:
                            currp = purchases[tsold]
                            currw[tsold] = 1
                if (
                    ((avg + margin) <= currp   and not shortsell) or
                    ((currp - margin) <= avg and shortsell)
                ):
                    break
    return ret

print(compute_gt(states[3:], 'PEARLS', [1, 2], [1, 2, 4, 8, 16, 20], [1,2,5,10,15,30], shortsell=False))
print(compute_gt(states[3:], 'PEARLS', [1, 2], [1, 2], [1,2000], shortsell=True))

{1: 9998.0, 2: 9998.0, 4: 10001.0, 8: 10004.75, 16: 10005.0, 20: 10005.0}
{1: {1: {1: 0, 2: 0, 4: 0, 8: 0, 16: 0, 20: 0}, 2: {1: 1, 2: 1, 4: 1, 8: 0, 16: 0, 20: 0}, 5: {1: 1, 2: 1, 4: 1, 8: 0, 16: 0, 20: 0}, 10: {1: 1, 2: 1, 4: 1, 8: 0, 16: 0, 20: 0}, 15: {1: 1, 2: 1, 4: 1, 8: 1, 16: 0, 20: 0}, 30: {1: 1, 2: 1, 4: 1, 8: 1, 16: 0, 20: 0}}, 2: {1: {1: 0, 2: 0, 4: 0, 8: 0, 16: 0, 20: 0}, 2: {1: 1, 2: 1, 4: 1, 8: 0, 16: 0, 20: 0}, 5: {1: 1, 2: 1, 4: 1, 8: 0, 16: 0, 20: 0}, 10: {1: 1, 2: 1, 4: 1, 8: 0, 16: 0, 20: 0}, 15: {1: 1, 2: 1, 4: 1, 8: 1, 16: 0, 20: 0}, 30: {1: 1, 2: 1, 4: 1, 8: 1, 16: 0, 20: 0}}}
{1: 9996.0, 2: 9996.0}
{1: {1: {1: 0, 2: 0}}, 2: {1: {1: 0, 2: 0}}}


In [117]:
print(states[50][1].order_depths['PEARLS'].buy_orders)
states[50][1].order_depths['PEARLS'].sell_orders

maxb = 0
mins = 10000000
for i,state in enumerate(states):
    maxb = max(maxb, max(state[1].order_depths['PEARLS'].buy_orders.keys()))
    mins = min(mins, min(state[1].order_depths['PEARLS'].sell_orders.keys()))
    if min(state[1].order_depths['PEARLS'].sell_orders.keys()) < 10002:
        print(i)
print(maxb)
print(mins)

{9998: 3, 9996.0: 1.0, 9995.0: 22.0}
3
36
37
44
49
54
60
61
82
88
106
123
145
152
164
173
191
220
228
229
232
245
267
269
276
279
296
299
301
317
320
347
360
361
363
379
382
403
406
412
413
416
424
432
438
451
453
460
477
478
498
508
526
527
528
532
538
550
559
576
583
584
585
589
591
593
601
619
635
654
660
663
668
674
678
702
708
734
735
737
747
748
749
763
775
779
804
806
807
831
862
868
869
870
871
907
912
913
916
918
936
951
957
958
959
980
982
988
992
994
1004
1006
1014
1021
1031
1036
1037
1040
1044
1086
1106
1114
1155
1163
1164
1179
1190
1207
1232
1241
1242
1251
1255
1264
1267
1271
1278
1281
1294
1295
1297
1323
1330
1375
1380
1382
1394
1405
1408
1410
1414
1425
1465
1481
1482
1514
1524
1530
1533
1553
1569
1580
1587
1590
1594
1630
1646
1650
1656
1690
1700
1702
1703
1716
1729
1752
1765
1766
1773
1800
1803
1823
1828
1832
1834
1864
1867
1872
1882
1889
1897
1898
1900
1902
1930
1952
1964
1970
1974
1995
10002
9998
